In [2]:
import numpy as np
from collections import deque

In [9]:
PUZZLE_DIM = 3

In [19]:
def generate_initial_state():
    """
    Generate a random solvable initial state for an n x n puzzle.

    Returns:
        tuple: A random, solvable initial state.
    """
    goal_state = tuple(range(1, PUZZLE_DIM * PUZZLE_DIM)) + (0,)
    while True:
        state = list(goal_state)
        np.random.shuffle(state)
        state = tuple(state)
        if is_solvable(state):
            return state

In [20]:
def is_solvable(state):
    """
    Check if a given puzzle state is solvable.

    Args:
        state (tuple): The current puzzle state.

    Returns:
        bool: True if the state is solvable, False otherwise.
    """
    # Count inversions
    inversion_count = 0
    state_list = [tile for tile in state if tile != 0]  # Exclude the blank tile
    for i in range(len(state_list)):
        for j in range(i + 1, len(state_list)):
            if state_list[i] > state_list[j]:
                inversion_count += 1

    if PUZZLE_DIM % 2 == 1:  # Odd grid
        return inversion_count % 2 == 0
    else:  # Even grid
        # Find the row of the blank tile (0), counting from the bottom
        blank_row_from_bottom = PUZZLE_DIM - (state.index(0) // PUZZLE_DIM)
        if blank_row_from_bottom % 2 == 0:  # Blank on an even row from bottom
            return inversion_count % 2 == 1
        else:  # Blank on an odd row from bottom
            return inversion_count % 2 == 0

In [65]:
n = PUZZLE_DIM

def get_neighbors(state):
    """Generate all possible moves from the current state."""
    zero_idx = state.index(0)
    neighbors = []
    zero_row, zero_col = divmod(zero_idx, n)
    # Define movement (up, down, left, right)
    moves = [(-1, 0, "U"), (1, 0, "D"), (0, -1, "L"), (0, 1, "R")]  # row, col adjustments
    for dr, dc, m in moves:
        new_row, new_col = zero_row + dr, zero_col + dc
        if 0 <= new_row < n and 0 <= new_col < n:
            new_idx = new_row * n + new_col
            # Swap empty tile with the target tile
            new_state = list(state)
            new_state[zero_idx], new_state[new_idx] = new_state[new_idx], new_state[zero_idx]
            neighbors.append((tuple(new_state), m))
    return neighbors

# Breadth-First Search

In [66]:
def bfs_solve_puzzle(start, goal):
    """
    Solves the n^2-1 puzzle using BFS.

    Args:
        start (tuple): The initial state of the puzzle.
        goal (tuple): The goal state of the puzzle.

    Returns:
        list: The solution path as a list of states, or None if no solution exists.
    """

    # BFS setup
    queue = deque([(start, [])])
    visited = set()

    while queue:
        current, path = queue.popleft()
        if current == goal:
            return path

        visited.add(current)
        for neighbor, move in get_neighbors(current):
            if neighbor not in visited and neighbor not in queue:
                queue.append((neighbor, path + [move]))

    return None

In [69]:
# state = generate_initial_state()
state = (1, 0, 2, 4, 5, 3, 7, 8, 6)

goal_state = tuple(range(1, PUZZLE_DIM * PUZZLE_DIM)) + (0,)

print("Initial state:")
print(state)
print()

path = bfs_solve_puzzle(state, goal_state)

print("Path to goal:")
print(path)
print()

Initial state:
(1, 0, 2, 4, 5, 3, 7, 8, 6)

Path to goal:
['R', 'D', 'D']



# Depth-First Search

In [68]:
def dfs_solve_puzzle(start, goal, depth_limit=None):
    """
    Solves the n^2-1 puzzle using DFS and returns the sequence of moves.

    Args:
        start (tuple): The initial state of the puzzle.
        goal (tuple): The goal state of the puzzle.
        depth_limit (int): Optional maximum depth for limited DFS.

    Returns:
        list: A list of moves (e.g., ["U", "L"]) to solve the puzzle, or None if no solution is found.
    """

    # Stack setup for DFS: each entry is (current_state, moves_so_far, current_depth)
    stack = [(start, [], 0)]
    visited = set()

    while stack:
        current_state, path, depth = stack.pop()

        # Check if we've reached the goal
        if current_state == goal:
            return path

        # Mark state as visited
        visited.add(current_state)

        # Stop exploring if depth limit is reached (for iterative deepening)
        if depth_limit is not None and depth >= depth_limit:
            continue

        # Explore neighbors (in reverse order for stack behavior to mimic DFS)
        for neighbor_state, move in reversed(get_neighbors(current_state)):
            if neighbor_state not in visited:
                stack.append((neighbor_state, path + [move], depth + 1))

    return None  # No solution

In [70]:
# state = generate_initial_state()
state = (1, 0, 2, 4, 5, 3, 7, 8, 6)

goal_state = tuple(range(1, PUZZLE_DIM * PUZZLE_DIM)) + (0,)

print("Initial state:")
print(state)
print()

path = dfs_solve_puzzle(state, goal_state)

print("Path to goal:")
print(path)
print()

Initial state:
(1, 0, 2, 4, 5, 3, 7, 8, 6)

Path to goal:
['D', 'D', 'L', 'U', 'U', 'R', 'D', 'D', 'L', 'U', 'U', 'R', 'D', 'D', 'L', 'U', 'U', 'R', 'D', 'D', 'L', 'U', 'U', 'R', 'D', 'D', 'L', 'U', 'R', 'U', 'L', 'D', 'D', 'R', 'U', 'U', 'L', 'D', 'D', 'R', 'U', 'U', 'L', 'D', 'D', 'R', 'U', 'U', 'L', 'D', 'D', 'R', 'U', 'U', 'L', 'D', 'D', 'R', 'R', 'U', 'U', 'L', 'D', 'D', 'L', 'U', 'U', 'R', 'D', 'D', 'L', 'U', 'U', 'R', 'D', 'D', 'L', 'U', 'U', 'R', 'D', 'D', 'L', 'U', 'U', 'R', 'D', 'D', 'L', 'U', 'R', 'U', 'L', 'D', 'D', 'R', 'U', 'U', 'L', 'D', 'D', 'R', 'U', 'U', 'L', 'D', 'D', 'R', 'U', 'U', 'L', 'D', 'D', 'R', 'U', 'U', 'L', 'D', 'D', 'R', 'R', 'U', 'U', 'L', 'D', 'D', 'L', 'U', 'U', 'R', 'D', 'D', 'L', 'U', 'U', 'R', 'D', 'D', 'L', 'U', 'U', 'R', 'D', 'D', 'L', 'U', 'U', 'R', 'D', 'D', 'L', 'U', 'R', 'U', 'L', 'D', 'D', 'R', 'U', 'U', 'L', 'D', 'D', 'R', 'U', 'U', 'L', 'D', 'D', 'R', 'U', 'U', 'L', 'D', 'D', 'R', 'U', 'U', 'L', 'D', 'D', 'R', 'R', 'U', 'U', 'L', 'D', 'D', '

## Iterative Deepening Depth-First Search

In [75]:
def iddfs_solve_puzzle(start, goal, max_depth):
    for depth_limit in range(max_depth + 1):
        solution = dfs_solve_puzzle(start, goal, depth_limit=depth_limit)
        if solution is not None:
            return solution
    return None

In [76]:
# state = generate_initial_state()
state = (1, 0, 2, 4, 5, 3, 7, 8, 6)

goal_state = tuple(range(1, PUZZLE_DIM * PUZZLE_DIM)) + (0,)

print("Initial state:")
print(state)
print()

path = iddfs_solve_puzzle(state, goal_state, 5)

print("Path to goal:")
print(path)
print()

Initial state:
(1, 0, 2, 4, 5, 3, 7, 8, 6)

Path to goal:
['R', 'D', 'D']



# Uniform Cost Search

In [77]:
import heapq

def ucs_solve_puzzle(start, goal):
    # Priority queue: each entry is (cost, current_state, moves_so_far)
    pq = []
    heapq.heappush(pq, (0, start, []))
    visited = set()

    while pq:
        cost, current_state, moves = heapq.heappop(pq)

        # Check if we've reached the goal
        if current_state == goal:
            return moves

        # Skip already visited states
        if current_state in visited:
            continue
        visited.add(current_state)

        # Explore neighbors
        for neighbor_state, move in get_neighbors(current_state):
            if neighbor_state not in visited:
                # Uniform cost increment: Each move costs 1
                heapq.heappush(pq, (cost + 1, neighbor_state, moves + [move]))

    return None  # No solution

In [79]:
# state = generate_initial_state()
state = (1, 0, 2, 4, 5, 3, 7, 8, 6)

goal_state = tuple(range(1, PUZZLE_DIM * PUZZLE_DIM)) + (0,)

print("Initial state:")
print(state)
print()

path = ucs_solve_puzzle(state, goal_state)

print("Path to goal:")
print(path)
print()

Initial state:
(1, 0, 2, 4, 5, 3, 7, 8, 6)

Path to goal:
['R', 'D', 'D']



# A* Search

In [82]:
import heapq

def a_star_solve_puzzle(start, goal):
    """
    Solves the n^2-1 puzzle using A* and returns the sequence of moves.

    Args:
        start (tuple): The initial state of the puzzle.
        goal (tuple): The goal state of the puzzle.

    Returns:
        list: A list of moves (e.g., ["U", "L"]) to solve the puzzle.
    """

    def manhattan_distance(state):
        """Calculates the Manhattan distance heuristic."""
        distance = 0
        for i, tile in enumerate(state):
            if tile == 0:  # Skip the empty tile
                continue
            goal_row, goal_col = divmod(tile - 1, n)
            current_row, current_col = divmod(i, n)
            distance += abs(goal_row - current_row) + abs(goal_col - current_col)
        return distance

    # Priority queue: each entry is (f_score, g_score, current_state, moves_so_far)
    pq = []
    heapq.heappush(pq, (manhattan_distance(start), 0, start, []))
    visited = set()

    while pq:
        f_score, g_score, current_state, moves = heapq.heappop(pq)

        # Check if we've reached the goal
        if current_state == goal:
            return moves

        # Skip already visited states
        if current_state in visited:
            continue
        visited.add(current_state)

        # Explore neighbors
        for neighbor_state, move in get_neighbors(current_state):
            if neighbor_state not in visited:
                # g_score + 1 is the new path cost; add heuristic for f_score
                new_g_score = g_score + 1
                new_f_score = new_g_score + manhattan_distance(neighbor_state)
                heapq.heappush(pq, (new_f_score, new_g_score, neighbor_state, moves + [move]))

    return None  # No solution

In [84]:
state = generate_initial_state()
# state = (1, 0, 2, 4, 5, 3, 7, 8, 6)

goal_state = tuple(range(1, PUZZLE_DIM * PUZZLE_DIM)) + (0,)

print("Initial state:")
print(state)
print()

path = a_star_solve_puzzle(state, goal_state)

print("Path to goal:")
print(path)
print()

Initial state:
(7, 1, 6, 5, 8, 3, 4, 2, 0)

Path to goal:
['L', 'U', 'R', 'D', 'L', 'L', 'U', 'U', 'R', 'D', 'D', 'L', 'U', 'R', 'R', 'U', 'L', 'D', 'D', 'R']

